# 2048 Keras

In [1]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input, concatenate, BatchNormalization, Activation
from keras.optimizers import Adadelta
import numpy as np

BATCH_SIZE = 128
NUM_EPOCHS = 15

Using TensorFlow backend.


In [2]:
OUT_SHAPE = (4,4)
CAND = 16
map_table = {2**i : i for i in range(1,CAND)}
map_table[0] = 0
vmap = np.vectorize(lambda x: map_table[x])

def grid_one(arr):
    ret = np.zeros(shape=OUT_SHAPE+(CAND,),dtype=bool)  # shape = (4,4,16)
    for r in range(OUT_SHAPE[0]):
        for c in range(OUT_SHAPE[1]):
            ret[r,c,arr[r,c]] = 1
    return ret

In [ ]:
import csv
data = []
with open("./train/train1M_1.csv") as f:
    for line in f:
        piece = eval(line)
        data.append(piece)

In [ ]:
data = np.array(data)

In [ ]:
x = np.array([ grid_one(piece[:-1].reshape(4,4)) for piece in data ])
y = keras.utils.to_categorical(data[:,-1], 4)

In [ ]:
sep = 1100000
x_train = x[:sep]
x_test = x[sep:]
y_train = y[:sep]
y_test = y[sep:]

In [ ]:
x_test.shape

In [ ]:
model = keras.models.load_model('model_k.h5')

In [ ]:
# train , validation_data=(x_test,y_test)
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1, verbose=1)

In [ ]:
score_test = model.evaluate(x_test,y_test,verbose=0)
print('Testing loss: %.4f, Testing accuracy: %.2f' % (score_test[0],score_test[1]))

In [ ]:
model.save('model_k.h5')  # creates a HDF5 file 'my_model.h5'

# 在线学习

In [1]:
import keras
from keras.models import Model
import numpy as np
import pickle
import datetime
import time
import _thread
import random
from collections import namedtuple
from game2048.game import Game
from game2048.expectimax import board_to_move

OUT_SHAPE = (4,4)
CAND = 16
map_table = {2**i : i for i in range(1,CAND)}
map_table[0] = 0
vmap = np.vectorize(lambda x: map_table[x])

def grid_one(arr):
    ret = np.zeros(shape=OUT_SHAPE+(CAND,),dtype=bool)  # shape = (4,4,16)
    for r in range(OUT_SHAPE[0]):
        for c in range(OUT_SHAPE[1]):
            ret[r,c,arr[r,c]] = 1
    return ret


class Guides:
    
    def __init__(self, cap):
        self.cap = cap
        self.mem = []
        self.pos = 0
        
    def push(self, xy):
        if len(self.mem) < self.cap:
            self.mem.append(None)
        self.mem[self.pos] = xy
        self.pos = (self.pos + 1) % self.cap
        
    def sample(self, batch_size):
        return random.sample(self.mem, batch_size)
    
    def ready(self,batch_size):
        return len(self.mem) >= batch_size
    
    def __len__(self):
        return len(self.mem)
    
    
class ModelWrapper:
    
    def __init__(self, model, cap):
        self.model = model
        self.mem = Guides(cap)
        #self.writer = tf.
        self.trainning_step = 0
        
    def predict(self, board):
        return model.predict(np.expand_dims(board,axis=0))
    
    def move(self, game):
        ohe_board = grid_one(vmap(game.board))        
        self.mem.push((ohe_board, board_to_move(game.board)))
        game.move(self.predict(ohe_board).argmax())
        
    def train(self, batch):
        if self.mem.ready(batch):
            guides = self.mem.sample(batch)
            X = []
            Y = []
            for guide in guides:
                X.append(guide[0])
                ohe_action = [0]*4
                ohe_action[guide[1]] = 1
                Y.append(ohe_action)
            loss, acc = self.model.train_on_batch(np.array(X), np.array(Y))
            print('#%d \t loss:%.3f \t acc:%.3f'%(self.trainning_step, float(loss), float(acc)))
            self.trainning_step += 1

Using TensorFlow backend.


Loaded expectmax lib for 2048: /home/faymek/2048-api/game2048/expectimax/bin/2048.so


# 数据生成线程

In [20]:
gen()

score: 512	score: 1024	

In [13]:
def gen():
    model_data = keras.models.load_model('model_last.h5')
    MEMORY = 1024
    mw_data = ModelWrapper(model_data,MEMORY)
    game = Game(4, random=False)
    for i in range(MEMORY):
        if game.end:
            print('score:',game.score, end='\t')
            game = Game(4, random=False)
        mw_data.move(game)
    filename = "mem/"+datetime.datetime.now().strftime('%y%m%d_%H%M%S')+".pickle"
    pickle.dump(mw_data.mem.mem,open(filename,"wb"))
    mem_list.append(filename)
    mw.NEWDATA += 1

In [22]:
gen()

score: 512	score: 256	score: 512	

In [14]:
model = keras.models.load_model('model.h5')
MEMORY = 1024
mw = ModelWrapper(model,MEMORY)

In [17]:
mw.NEWDATA = 1
mem_list = []
maxiter = 0

In [18]:
files = random.sample(mem_list,maxiter)
print(files)
for file in files:
    mw.mem.mem = pickle.load(open(file,"rb"))
    mw.train(1024)
if mw.NEWDATA==0:
    maxiter+=1
    while (mw.NEWDATA==0):
        print("sleep 5")
        time.sleep(5)
print("maxiter",maxiter)
mw.NEWDATA=0
model.save("model_last.h5")

In [21]:
_thread.start_new_thread( gen, () )

139736160864000

Unhandled exception in thread started by <function gen at 0x7f173f372840>


In [54]:
import time
print(1)
time.sleep(1)
print(2)

1
2


# 训练过程

In [7]:
#while True:

random.sample(mem_list,maxiter)
for file in mem_list:
    
    

In [9]:
pickle.load(open("tttt.pickle","rb"))

In [17]:
filename = "mem/"+datetime.datetime.now().strftime('%y%m%d_%H%M%S')+".csv"
filename

'mem/190110_225209.csv'

In [10]:
import datetime

array([[1, 2, 3],
       [4, 5, 6]])

In [2]:
model = keras.models.load_model('model.h5')

In [4]:
MEMORY = 1024
BATCH = 256
mw = ModelWrapper(model,MEMORY)

In [5]:
while True:
    game = Game(4, random=False)
    while not game.end:
        mw.move(game)
    print('score:',game.score, end='\t')
    mw.train(BATCH)
    if(mw.trainning_step%10==0):
        model.save('modelOL.h5')

score: 1024	#0 	 loss:1.910 	 acc:0.508
score: 512	#1 	 loss:1.060 	 acc:0.625
score: 512	#2 	 loss:1.080 	 acc:0.590
score: 1024	#3 	 loss:1.717 	 acc:0.488
score: 256	#4 	 loss:1.998 	 acc:0.480
score: 512	#5 	 loss:1.627 	 acc:0.539
score: 128	#6 	 loss:1.367 	 acc:0.520


KeyboardInterrupt: 

In [5]:
# 先填满内存
while not mw.mem.ready(BATCH):
    game = Game(4, random=False)
    while not game.end:
        mw.move(game)

In [ ]:
def generate_data():
    while True:
        game = Game(4, random=False)
        while not game.end:
            mw.move(game)
        print('score:',game.score, end='\t')
        X = []
        Y = []
        for guide in mw.mem.mem:
            X.append(guide.state)
            ohe_action = [0]*4
            ohe_action[guide.action] = 1
            Y.append(ohe_action)
        yield np.array(X), np.array(Y)
            
model.fit_generator(generate_data(), steps_per_epoch=1, epochs=20)

In [46]:
len(mw.mem.mem)

7450

# 测试分数

In [2]:
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent
# from game2048.displayer import Displayer
display1 = Display()
display2 = IPythonDisplay()

In [ ]:
map_table = {2**i : i for i in range(1,CAND)}
map_table[0] = 0

class MyAgent(Agent):

    def __init__(self, game, display=None):
        super().__init__(game, display)
        
    def step(self):
        x0 = np.array([ grid_one(vmap(game.board)) ])
        preds = list(model.predict(x0))
        direction = np.argmax(preds[0])
        return direction


In [ ]:
game = Game(4, random=False)
agent = MyAgent(game, display=display1)
agent.play()
game.score

In [ ]:
scores = []
for i in range(10):
    game = Game(4, random=False)
    agent = MyAgent(game, display=display1)
    agent.play()
    scores.append(game.score)
scores

In [45]:
from game2048.game import Game
from game2048.expectimax import board_to_move
vd2 = np.vectorize(lambda x: int(x/2))

In [211]:
game = Game(4, random=False, score_to_win=1024)
#game.enable_rewrite_board = True
#cur_board = game.board
count = 0
err = []
while not game.end:
    d = board_to_move(game.board)
    count += 1
    cur_board = game.board
    ds = []
    for i in range(4):
        ds.append(board_to_move(cur_board))
        cur_board = np.rot90(cur_board)

    if not (0 in ds and 1 in ds and 2 in ds and 3 in ds):
        print(game.board)
    game.move(d)
print(count)

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  4.]
 [ 0.  0.  0.  2.]]
[[ 0.  0.  0.  0.]
 [ 0.  0.  4.  0.]
 [ 0.  0.  0.  4.]
 [ 4.  0.  0.  2.]]
[[ 4.  0.  8.  4.]
 [ 0.  0.  0.  2.]
 [ 0.  0.  0.  2.]
 [ 0.  0.  0.  0.]]
[[ 4.  8.  4.  0.]
 [ 2.  0.  0.  0.]
 [ 2.  0.  0.  0.]
 [ 0.  0.  0.  2.]]
[[ 4.  8.  4.  0.]
 [ 2.  0.  0.  0.]
 [ 2.  0.  0.  2.]
 [ 2.  0.  0.  0.]]
[[ 4.  2.  0.  0.]
 [ 2.  0.  0.  0.]
 [ 4.  0.  0.  0.]
 [ 2.  8.  4.  4.]]
[[   0.    0.    0.    4.]
 [   2.    0.    0.    0.]
 [  16.    4.    0.    0.]
 [ 256.    4.    4.    0.]]
[[   0.    0.    2.    0.]
 [   4.    0.    0.    0.]
 [  16.    0.    0.    0.]
 [ 256.    8.    4.    4.]]
[[   8.    8.    0.    0.]
 [  16.    0.    0.    0.]
 [  32.   16.    4.    2.]
 [ 256.   32.    4.    0.]]
[[   0.    0.    2.    0.]
 [   0.    0.    4.    0.]
 [   4.    8.    2.    2.]
 [ 256.  128.   64.   32.]]
[[   0.    0.    2.    4.]
 [   0.    0.    4.    2.]
 [   8.    0.   64.    4.]
 [   8.   32.   32.  

In [141]:
print(cur_board)
print(board_to_move(cur_board))
print(vd2(cur_board))
print(board_to_move(vd2(cur_board)))

[[    4.     0.     0.     0.]
 [    4.     0.     0.     0.]
 [   16.     8.     8.     0.]
 [   32.   128.   512.  1024.]]
0
[[  2   0   0   0]
 [  2   0   0   0]
 [  8   4   4   0]
 [ 16  64 256 512]]
1


In [118]:
m2 = np.ones((4,4))*2

In [202]:
game = Game(4, random=True)
cur_board = game.board
print(cur_board)
for i in range(4):
    print(board_to_move(cur_board))
    cur_board = np.rot90(cur_board)


[[ 16 512 256   8]
 [256  64   4  64]
 [512 256  32   8]
 [ 32   8  64  64]]
0
3
0
3


In [180]:
32 in game.board

True

585